In [1]:
from platform import python_version
print(python_version())

from builtins import exec
exec(open("./imports_functions.py").read())

import h5py

%matplotlib inline

3.10.4


In [5]:
if not os.path.exists("goard"):
    print("Downloading RSC data ...")
    os.makedirs("goard")
    # resp = wget.download("https://datadryad.org/stash/downloads/file_stream/1185004", "goard/7739750.zip")
    # print("... Done: "+resp)

    if os.path.exists("goard/twoPhotonData.mat"):
        print("Done.")
    else:
        print("not found")
else:
    print("Loading RSC data ...")
    twoPhotonData = h5py.File('./goard/twoPhotonData.mat','r')
    print("All data available.")

Loading RSC data ...
All data available.


### Data preprocessing

e `twoPhotonData.mat`.



In [8]:
cellResponses = np.array( twoPhotonData.get('cellData/cellResponses') ) # Get cell responses
# print(cellResponses.shape) # (4, 56, 90, 7770) reversed to the description

dff_traces = np.swapaxes(cellResponses.reshape(-1, cellResponses.shape[-1]), 0, 1) # by cell flattened over the other dimensions to gather enough activations
mask = np.isnan(dff_traces) # NaN was used to fill missing data
dff_traces[mask] = 0.0 # we assign 0 df/f
# print("dff_traces",dff_traces.shape)
ophys_cell_ids = np.arange(0,dff_traces.shape[0])
ophys_cell_indexes = ophys_cell_ids

print("... getting reliable cells over trials")
# reliableCells: [5194×1 double] ---> indices of the 5194 cells with reliable responses
reliableCells = np.squeeze( np.array(twoPhotonData.get('cellData/reliableCells'),dtype=np.int) ) # Get ids of cells reliably participating to the task
# print("reliableCells",reliableCells.shape)
reliableCells -= 1 # matlab indices start from 1
dff_traces = dff_traces[reliableCells] # keep only reliable cells
print("dff_traces",dff_traces.shape)


... getting reliable cells over trials


/tmp/ipykernel_12061/729084863.py:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  reliableCells = np.squeeze( np.array(twoPhotonData.get('cellData/reliableCells'),dtype=np.int) ) # Get ids of cells reliably participating to the task


dff_traces (5194, 20160)


We should separate the 29 sessions
